### - Задача 1.
Выберите датасет, который имеет отношение к вашей области интересов или исследований. Датасет должен содержать неструктурированные данные, требующие разметки для решения конкретной задачи, например, анализа настроений или распознавания именованных сущностей.

In [2]:
!pip install wget

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9680 sha256=743c8746c3620297326a4df54d96b75f15d1b0b9d9d9ef55e43b9036bdfe52be
  Stored in directory: c:\users\olesya\appdata\local\pip\cache\wheels\46\78\0e\8e5e2b500f83a682c8d7e7ce820638cf99faa894a662f71cf0
Successfully built wget


In [283]:
import pandas as pd
import nltk
# Загружаем предобученную модель токенизации текста
nltk.download('punkt')
import csv
import os
import wget
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import matplotlib
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline # конвеер обработки данных
from sklearn.metrics import accuracy_score, classification_report # оценка производительности модели

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Olesya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [314]:
# Загрузим датасет с отзывами на телефоны
df = pd.read_csv('C:\GB\DataEngineer\Data_collection\HW\9\data_copy.csv', engine='python', on_bad_lines='skip')
df

,Unnamed: 0,Review,Rating
0,0,3D Touch просто восхитительная вещь! Заряд дер...,5
1,1,"Отключается при температуре близкой к нулю, не...",4
2,2,"В Apple окончательно решили не заморачиваться,...",3
3,3,Постарался наиболее ёмко и коротко описать все...,4
4,4,Достойный телефон. Пользоваться одно удовольст...,5
...,...,...,...
4685,4685,"Покупкой довольна!!! Android 6.0, LTE, IPS-дис...",5
4686,4686,Это самый лучший мобильный в своей ценовой кат...,5
4687,4687,Рассматривали в подарок супруге 3 варианта. Эт...,5
4688,4688,"Понравился дизайн телефона, оказался даже очен...",5


In [112]:
# Загрузим тональный словарь
url = 'https://raw.githubusercontent.com/dkulagin/kartaslov/master/dataset/kartaslovsent/kartaslovsent.csv'
path = 'C:\GB\DataEngineer\Data_collection\HW\9\kartaslovsent.csv'
if not os.path.isfile("kartaslovsent.csv"):
    wget.download(url, out=path)

100% [..........................................................................] 2655438 / 2655438

In [113]:
df_dictionary = pd.read_csv(path, sep=';', on_bad_lines='skip')

In [114]:
df_dictionary

,term,tag,value,pstv,ngtv,neut,dunno,pstvNgtvDisagreementRatio
0,абажур,NEUT,0.08,0.185,0.037,0.580,0.198,0.00
1,аббатство,NEUT,0.10,0.192,0.038,0.578,0.192,0.00
2,аббревиатура,NEUT,0.08,0.196,0.000,0.630,0.174,0.00
3,абзац,NEUT,0.00,0.137,0.000,0.706,0.157,0.00
4,абиссинец,NEUT,0.28,0.151,0.113,0.245,0.491,0.19
...,...,...,...,...,...,...,...,...
46122,ёмкость,NEUT,0.00,0.167,0.000,0.690,0.143,0.00
46123,ёрзать,NGTV,-0.54,0.050,0.446,0.397,0.107,0.00
46124,ёрничать,NGTV,-0.79,0.078,0.529,0.236,0.157,0.00
46125,ёрш,NEUT,0.16,0.224,0.072,0.576,0.128,0.00




- Задача 3.
Выполните разметку вручную отдельного подмножества выбранного датасета с помощью выбранного вами инструмента разметки.

In [142]:
# разделим датафрем на 2 части

part_rule, part_manually = train_test_split(df['Review'], train_size=int(len(df)*0.01), random_state=42)

In [143]:
# Преобразуем полученые датасеты в датафреймы Pandas
df_part_manually  = pd.DataFrame(part_rule)
df_part_rule  = pd.DataFrame(part_manually)

In [144]:
df_part_manually.info()

<class 'pandas.core.frame.DataFrame'>
Index: 46 entries, 1363 to 860
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  46 non-null     object
dtypes: object(1)
memory usage: 736.0+ bytes


In [145]:
df_part_rule.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4644 entries, 1550 to 1478
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  4638 non-null   object
dtypes: object(1)
memory usage: 72.6+ KB


### - Задача 2.
Выполните разметку на основе правил (rule-based labeling) на подмножестве выбранного датасета. Разработайте и реализуйте набор правил или условий, которые позволят автоматически присваивать метки данным на основе определенных шаблонов или критериев.


In [146]:
# Для разметки на основе правил создадим функцию определения уровня отзыва по упоминанию положительных или отрицательных слов
def set_sentiment(text, df_td=df_dictionary):
    # Токенизация текста на отдельные слова
    words = nltk.word_tokenize(str(text).lower())
    sum_value = 0  # Сумма всех зачений весов найденных слов
    
    for word in words:
        if len(df_td[df_td.term == word]):
            df_filtered = df_td[df_td['term'] == word]
            value = df_filtered['value'].iloc[0]
            sum_value += value
 
    return sum_value   


In [147]:
# Функция определяет метку отзыва на основе уровня отзыва
def set_label(score):
    if score < -0.4:
        return 'negative'  
    elif score >= 0.50:
        return 'positive' 
    else:
        return 'neutral'

Присвоим метки отзыва на основе правил

In [148]:

df_part_rule['sentiment'] = df_part_rule['Review'].apply(set_sentiment)
df_part_rule['sentiment_label'] = df_part_rule['sentiment'].apply(set_label)

In [149]:
df_part_rule.head()

,Review,sentiment,sentiment_label
1550,Не советую этот телефон.,0.34,neutral
4400,Вышел Android 5.1.1 (сборка 18.6.A.0.175). Нем...,0.73,positive
2846,"Жду того момента, когда куплю новую трубку и в...",2.01,positive
584,Аппарат классный. Спрыгнул на него с пятого Ай...,3.93,positive
2339,"Самый худший телефон, который у меня был!!!!!",-0.66,negative


In [151]:
df_part_rule['sentiment_label'].value_counts()

sentiment_label
positive    3543
neutral      949
negative     152
Name: count, dtype: int64

In [254]:
df_part_rule.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4644 entries, 1550 to 1478
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Review           4638 non-null   object 
 1   sentiment        4644 non-null   float64
 2   sentiment_label  4644 non-null   object 
dtypes: float64(1), object(2)
memory usage: 145.1+ KB


In [267]:
#удалим строки с пустыми отзывами
df_part_rule['Review'].replace('', np.nan, inplace=True)
df_part_rule.dropna(subset=['Review'], inplace=True)
df_part_rule.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4638 entries, 1550 to 1478
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Review           4638 non-null   object 
 1   sentiment        4638 non-null   float64
 2   sentiment_label  4638 non-null   object 
dtypes: float64(1), object(2)
memory usage: 144.9+ KB


C:\Users\Olesya\AppData\Local\Temp\ipykernel_25000\1542439404.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_part_rule['Review'].replace('', np.nan, inplace=True)


### - Задача 3.
Выполните разметку вручную отдельного подмножества выбранного датасета с помощью выбранного вами инструмента разметки.

In [152]:
#Сохраним датафрем в CSV файл
df_part_manually.to_csv('part_manually.csv', index=False)

Запускаем label-studio, загружаем файл и вручную размечаем его, затем выгружаем

In [205]:
df_part_manually = pd.read_csv('C:\GB\DataEngineer\Data_collection\HW\9\project-3-at-2024-05-27-20-53-6f027395.csv', engine='python', on_bad_lines='skip')

In [207]:
df_part_manually.head()

,Review,annotation_id,annotator,created_at,id,lead_time,sentiment,updated_at
0,"Это мой первый опыт эксплуатации смартфона, ра...",23,1,2024-05-27T17:44:18.842265Z,58,7.653,positive,2024-05-27T17:44:18.842265Z
1,"Увы, главную детскую болезнь за все время выпу...",24,1,2024-05-27T17:44:31.161330Z,59,8.481,negative,2024-05-27T17:44:31.161330Z
2,"Не когда не писал отзыв, но сегодня он меня до...",25,1,2024-05-27T17:44:44.437715Z,60,10.174,negative,2024-05-27T17:44:44.437715Z
3,Некоторые отзывы на ЯМ не достойны внимания - ...,26,1,2024-05-27T17:44:53.683525Z,61,6.629,neutral,2024-05-27T17:44:53.683525Z
4,"в целом,покупкой довольна,берите,цена все мале...",27,1,2024-05-27T17:45:02.145406Z,62,24.413,positive,2024-05-27T17:45:29.441202Z


In [208]:
# Оставим только нужные столбцы
df_part_manually = df_part_manually[['Review', 'sentiment']].rename(columns={'sentiment': 'sentiment_label'})

In [209]:
df_part_manually.head()

,Review,sentiment_label
0,"Это мой первый опыт эксплуатации смартфона, ра...",positive
1,"Увы, главную детскую болезнь за все время выпу...",negative
2,"Не когда не писал отзыв, но сегодня он меня до...",negative
3,Некоторые отзывы на ЯМ не достойны внимания - ...,neutral
4,"в целом,покупкой довольна,берите,цена все мале...",positive



### - Задача 4.
Объедините данные, размеченные вручную, с данными, размеченными на основе правил. Объедините два подмножества размеченных данных в один набор данных, сохранив при этом соответствующую структуру и целостность.

In [268]:
df_review = pd.concat([
    df_part_manually,
    df_part_rule,
], axis=0)

del df_review['sentiment']

In [269]:
df_review.head()

,Review,sentiment_label
0,"Это мой первый опыт эксплуатации смартфона, ра...",positive
1,"Увы, главную детскую болезнь за все время выпу...",negative
2,"Не когда не писал отзыв, но сегодня он меня до...",negative
3,Некоторые отзывы на ЯМ не достойны внимания - ...,neutral
4,"в целом,покупкой довольна,берите,цена все мале...",positive


In [270]:
df_review.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4684 entries, 0 to 1478
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Review           4684 non-null   object
 1   sentiment_label  4684 non-null   object
dtypes: object(2)
memory usage: 109.8+ KB


In [285]:
# Посмотрим, сколько каких отзывов и запишем датасет в файл
df_review.to_csv('df_review.csv')
df_review['sentiment_label'].value_counts()

sentiment_label
positive    3570
neutral      955
negative     159
Name: count, dtype: int64

### - Задача 5.
Обучите модель машинного обучения, используя объединенный набор размеченных данных. Разделите датасет на обучающий и тестовый наборы и используйте обучающий набор для обучения модели.

In [286]:
# Перемешивание данных
df_review = shuffle(df_review)

In [287]:
le = LabelEncoder()
# Преобразуем в датафрейме буквеное обозначение в цифровое
df_review['sentiment_label'] = le.fit_transform(df_review['sentiment_label'].astype(str))

In [288]:
X_train, X_test, y_train, y_test = train_test_split(
df_review['Review'], # текст рецензии - будет использоваться как входные данные
df_review['sentiment_label'], # метка классов (положительный или отрицательный )
test_size=0.2, # доля данных которая попадет в тест (20%)
random_state=42 # Зерно генератора случайных чисел для воспроизводимости результатов
)

In [289]:
pipeline = Pipeline([
('tfidf', TfidfVectorizer(stop_words='english')), # векторизация текста, с исключением стоп слов англ языка
('clf', LogisticRegression(max_iter=1000)) # модель логистической регресии с увелечинным колличеством итераций до 1000
])

In [290]:
#Обучение модели на обучающем наборе данных
pipeline.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer(stop_words='english')),
                ('clf', LogisticRegression(max_iter=1000))])


### - Задача 6.
Оценить эффективность обученной модели на тестовом датасете. Используйте подходящие метрики оценки. Интерпретируйте результаты и проанализируйте эффективность модели в решении задачи разметки.

In [307]:
predictions = pipeline.predict(X_test) # предсказание модели на тестовых данных
accuracy = accuracy_score(y_test, predictions) # расчет точности модели
report = classification_report(y_test, predictions) # оценка качесвта предсказания

print(f"Точность модели {accuracy}")
print('Отчет по классификации')
print(report)

Точность модели 0.8516542155816436
Отчет по классификации
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        16
           1       0.81      0.39      0.53       181
           2       0.86      0.98      0.92       740

    accuracy                           0.85       937
   macro avg       0.55      0.46      0.48       937
weighted avg       0.83      0.85      0.82       937



C:\Users\Olesya\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Olesya\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Olesya\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Точность модели состовляет 85%. Думаю, можно повысить точность засчет увеличения объема данныъ, так как для анализа взяла 1/60 часть реального датасета, но и эту часть разметка по правилам заняла около получаса...